In [1]:
train <- read.csv(file='/root/hackerday/06_san_francisco_salary/train.csv', header=TRUE,sep=",")

In [2]:
test <- read.csv(file='/root/hackerday/06_san_francisco_salary/test.csv', header=TRUE,sep=",")

In [3]:
#3. xgboost --boosting algorithm
library(xgboost)

In [4]:
var = names(train[2:ncol(train)])

In [5]:
train$label = as.numeric(train$label)
train[var] =sapply(train[var], function(x) as.numeric(x))

In [ ]:
all_train = xgb.DMatrix(data=data.matrix(train[var]),label=(train$label))




#parameters to be passed in xgboost
param <- list(objective           = "multi:softprob", 
              booster = "gbtree",
              eta                 = 0.01, # 0.06, #0.01,0.005
              max_depth           = 10, #changed from default of 4,6,8,10,15,20
              subsample           = 0.7, #(.5,0.7,1)
              colsample_bytree    = 0.7, #(.5,0.7,1)
              min_child_weight=2,  ## 3/ Event rate - Rule of Thumb 
              num_class = 10
)

## Train the model with 2 fold cross validation
clf <- xgb.cv(params              = param, 
              data                = all_train, 
              nrounds             = 200, #300, #280, #125, #250, # changed from 300
              verbose             = 1,
              #early.stop.round    = 40,
              #watchlist           = watchlist,
              maximize            = FALSE,
              eval_metric="mlogloss",
              nfold=2
)

###### Use the entire training set using best parameters 
clf_best <- xgboost(params        = param, 
                    data                = all_train, 
                    nrounds             = 100, #300, #280, #125, #250, # changed from 300
                    verbose             = 1,
                    #early.stop.round    = 200,
                    #watchlist           = watchlist,
                    maximize            = FALSE,
                    eval_metric="mlogloss"
                    #nfold=3
)

imp_var_test <- names(test[1:ncol(test)])
test[imp_var_test] <- sapply(test[imp_var_test],function(x) as.numeric(x))
testDataMatrix<-xgb.DMatrix(data=data.matrix((test[imp_var_test])))
pred1 <- predict(clf_best,testDataMatrix)
pred = matrix(pred1, nrow=1)
pred = data.frame(t(pred))

pred2 <- data.frame(ImageId=1:nrow(test),Label=pred)
write.csv(pred2,"xgboost_bm.csv",row.names = F)
    
    
##neural network in H20
    
    #install.packages("h2o")
library(h2o)
## start a local h2o cluster
localH2O = h2o.init(max_mem_size = '6g', # use 6GB of RAM of *GB available
                    nthreads = -1) # use all CPUs (8 on my personal computer :3)

## MNIST data as H2O
train[,1] = as.factor(train[,1]) # convert digit labels to factor for classification
train_h2o = as.h2o(train)
test_h2o = as.h2o(test)


## set timer
s <- proc.time()

## train model
model =
  h2o.deeplearning(x = 2:785,  # column numbers for predictors
                   y = 1,   # column number for label
                   training_frame = train_h2o, # data in H2O format
                   activation = "RectifierWithDropout", # algorithm
                   input_dropout_ratio = 0.2, # % of inputs dropout
                   hidden_dropout_ratios = c(0.5,0.5), # % for nodes dropout
                   balance_classes = TRUE, 
                   hidden = c(100,100), # two layers of 100 nodes
                   momentum_stable = 0.99,
                   nesterov_accelerated_gradient = T, # use it for speed
                   epochs = 15) # no. of epochs


## print confusion matrix
h2o.confusionMatrix(model)

## print time elapsed
s - proc.time()

## classify test set
h2o_y_test <- h2o.predict(model, test_h2o)

## convert H2O format into data frame and  save as csv
df_y_test = as.data.frame(h2o_y_test)
df_y_test = data.frame(ImageId = seq(1,length(df_y_test$predict)), Label = df_y_test$predict)
write.csv(df_y_test, file = "submission-r-h2o.csv", row.names=F)

## shut down virutal H2O cluster
h2o.shutdown(prompt = F)

    
